# 一、线性回归实现(纯手撸)
## 1.生成数据集
### 使用函数部分
* **torch.normal**：返回一个张量，包含从给定参数means,std的*离散正态分布*中抽取随机数。 均值means是一个张量，包含每个输出元素相关的正态分布的均值。 std是一个张量，包含每个输出元素相关的正态分布的标准差。 均值和标准差的形状不须匹配，但每个张量的元素个数须相同。同时out可以指定输出张量的形状（shape）
* **torch.matmul**: 二维时等同于矩阵乘法

In [ ]:
import torch
import numpy as np
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)# 加上噪音
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
features, labels

## 2.生成数据集图像

In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(features[:,0],labels)

## 3.读取数据集
### 使用函数部分
* **random.shuffle()**: 用于将一个列表中的元素打乱顺序，值得注意的是使用这个方法不会生成新的列表，只是将原列表的次序打乱。
* **yield**:带yield的函数是一个<mark>生成器</mark>，而不是一个函数了，这个生成器有一个函数就是next函数，next就相当于“下一步”生成哪个数，这一次的next开始的地方是接着上一次的next停止的地方执行的，所以调用next的时候，生成器并不会从foo函数的开始执行，只是接着上一步停止的地方开始，然后遇到yield后，return出要生成的数，此步就结束。


In [ ]:
# 小批量随机读取
import random
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # indices代表一个随机序号列表
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        # 每次取出indices中的batch个序列表
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    #只取一组
    break

## 4.定义模型

In [ ]:
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

## 5.定义损失算法

In [ ]:
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

## 6.定义优化算法

In [ ]:
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():#异常处理
        for param in params:
            param -= lr * param.grad / batch_size#规范化步长
            param.grad.zero_()#梯度归零

## 7.训练模型

In [ ]:
from importlib.metadata import requires
import torch
import numpy as np
import random
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)# 加上噪音
    return X, y.reshape((-1, 1))
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # indices代表一个随机序号列表
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        # 每次取出indices中的batch个序列表
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():#异常处理
        for param in params:
            param -= lr * param.grad / batch_size#规范化步长
            param.grad.zero_()#梯度归零

batch_size = 10
true_w = torch.tensor([2, -3.4])
true_b = 4.2
w=torch.tensor([1.,-3.])
b=4.
features, labels = synthetic_data(true_w, true_b, 1000)
lr = 0.03
num_epochs = 3 # 迭代周期个数
net = linreg
loss = squared_loss
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features,w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

# 二、深度学习框架解决


## 生成数据集与上图类似

In [1]:
import torch
from torch.utils import data
import numpy as np
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)# 加上噪音
    return X, y.reshape((-1, 1))
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

e:\coding_setting\Miniconda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 读取数据集--使用深度学习框架
### 使用函数部分
* data.TensorDataset():TensorDataset 可以用来对 tensor 进行打包，就好像 python 中的 zip 功能。该类通过每一个 tensor 的第一个维度进行索引。因此，该类中的 tensor 第一维度必须相等. 另外：TensorDataset 中的参数必须是 tensor
* data.DataLoader就是用来包装所使用的数据，每次抛出批量大小为batch_size的数据，可选择是否随机打乱，它是一个迭代器
* nn:神经网络缩写
* Sequential:神经网络的构造容器，可以理解为放置多个层的大容器，当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。
* Linear：torch.nn.Linear(in_features, *输入的神经元个数* out_features, *输出神经元个数*
           bias=True  *是否包含偏置*)，默认会利用标准正态分布初始化w和b，也可以自己定义初始值。
* MSELoss: 返回样本损失平均值
* torch.optim.SGD: 小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值(学习率)，这里设置为0.03。


In [2]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))#迭代器执行一次

[tensor([[-6.0436e-01,  4.2205e-02],
         [-3.3791e-01, -1.7858e-01],
         [ 1.3887e+00, -1.0825e+00],
         [ 9.7460e-01,  3.4841e-01],
         [-1.7414e-03,  3.9982e-01],
         [ 7.7913e-01, -3.9115e-01],
         [ 1.9110e+00, -3.2618e-01],
         [ 1.1407e+00, -2.9752e-01],
         [ 5.4157e-01,  5.8803e-01],
         [ 1.8378e+00, -1.7873e-01]]),
 tensor([[ 2.8587],
         [ 4.1368],
         [10.6487],
         [ 4.9791],
         [ 2.8637],
         [ 7.0934],
         [ 9.1467],
         [ 7.4832],
         [ 3.2930],
         [ 8.4818]])]

In [3]:
# nn是神经网络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))# 输入特征数为2，输出特征数为1
# net[0].weight.data.normal_(0,0.01)# 训练时的初始权重赋值
# net[0].bias.data.fill_(0)# 训练时的初始偏差赋值


In [4]:
loss = nn.MSELoss()

In [5]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [6]:
#开始训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter: #取出数据集中的小批量数据
        l = loss(net(X) ,y) # 计算单次损失率
        trainer.zero_grad()# 梯度归零
        l.backward()# 计算梯度
        trainer.step()# 进行参数的优化
    l = loss(net(features), labels)# 一次优化结束后的损失率情况
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000238
epoch 2, loss 0.000103
epoch 3, loss 0.000104


In [8]:
# 三次优化完计算误差
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([[-0.0007,  0.0010]])
b的估计误差： tensor([4.7684e-06])


# 总结
## 线性回归的步骤总结
* 1.建立数据集（一般来说读取给定的csv文件）
* 2.读取数据集，使用TensorDataset打包，再使用DataLoader读取批量的数据。
* 3.定义神经网络，将每一层封装在容器Sequential中。
* 4.定义损失函数
* 5.定义训练模型并初始化
* 6.进行训练